#Final Project

Context
You are a conservation biologist in charge of managing habitats for a protected
reserve. Your reserve includes several distinct patches of intact forest that harbor
populations of the endangered Warbling Babbler. You are worried about the longterm
fate of these populations if the habitats remain disconnected with limited
movement of individuals between them (i.e., limited migration between the
populations), so youʼd like to use funds from your budget to establish habitat
corridors that will allow individuals to move between the patches. However, you also
know that the populations have some phenotypic differences (like color) and you are
wondering how the frequencies of the phenotypes, and the overall population sizes,
will change in the populations once they are connected.


In [37]:
# Import necessary modules
import matplotlib.pyplot as plt
import random

Your framework will include three types of classes: an Individual class, a Population class, and a Landscape class. Individuals exist in populations, and populations exist in a landscape. In each time step of your simulation, individuals can stay in the population where they started, or move to a
new population. The probabilities that individuals stay or leave are stored in a table called a dispersal matrix (more detail below).

In [1]:
class Individual:
    """Class to hold information on indiviuals."""# Add a docstring
        
    def __init__(self, id, phenotype):
        """The constructor for the individual class."""
        self.id = id
        self.phenotype = phenotype

    def setPhenotype(self, newPhenotype):
        self.phenotype = newPhenotype

In [6]:
testIndividual = Individual("LSU1", "purple")
print(testIndividual.phenotype)
testIndividual.setPhenotype("red")
print(testIndividual.phenotype)


purple
red


Population constructors should create a new list for individuals in that population,
and Populations should have methods to add and remove individuals, as well as to
calculate and print the frequency of phenotypes among individuals.

In [8]:
class Population:
    """Class to hold information on populations that is composed of individuals."""
    
    def __init__(self, id, popSize, phenotype): 
        """"The constructor for the population class."""
        self.id = id
        self.individuals = []
        for i in range(popSize):
            self.individuals.append(Individual("%s-%d" % (id, i+1), phenotype))

    def addIndividual(self, individual):
        self.individuals.append(individual)
    
    def removeIndividual(self, individual):
        self.individuals.remove(individual)
    
    def calculateFreq(self, phenotype):
        freq = 0
        for ind in self.individuals:
             if ind.phenotype == phenotype:
                 freq = freq + 1
        return freq
    
    def printFreq(self, phenotype):
        print("The %s phenotype frequency is %d" % (phenotype, self.calculateFreq(phenotype)))

In [12]:
testPopulation = Population("popA", 12, "purple")
for ind in testPopulation.individuals:
    print(ind.phenotype) 
testPopulation.printFreq("purple")

purple
purple
purple
purple
purple
purple
purple
purple
purple
purple
purple
purple
The purple phenotype frequency is 12


The dispersal matrix consists of
the probabilities that an individual disperses between populations, or stays in place,
each time step. For Warbling Babblers, these probabilities correspond to their
probability of moving in a week (Fig. 1). If an individual moves from one population
to another, you should remove it from the list of individuals in the population where
it started, and add it to the list of individuals in the population where it went. There
are multiple ways to use the probabilities in the dispersal matrix to decide if an
individual moves in each time step, but one function you may find helpful is
random.choices(…) from the random module.

Landscape constructors should create a new list for the populations in that landscape, and Landscapes should have a method that uses the probabilities in the dispersal matrix to determine if an individual moves or stays each time step. Either the Population or Landscape constructors should set the starting population sizes. The total number of individuals across all populations should stay constant, but individual populations may change size.

In [70]:
class Landscape:
    """Class to hold information on landscapes, which are composed of different populations."""
    
    def __init__(self, popSize=10, phenotypes=None, dispersalMatrix=None, simulationSteps=1): 
        """"The constructor for the Landscape class, taking a whole lot of arguments."""
        self.dispersalMatrix = dispersalMatrix
        self.simulationSteps = simulationSteps
        self.populations = []
        self.popAlias = []
        for i in range(len(phenotypes)):
            self.populations.append(Population('pop%s' % chr(65 + i), popSize, phenotypes[i]))
            self.popAlias.append('pop%s' % chr(65 + i))
    
    def migratePop(self):
        for i in range(len(self.populations)): #loop through the populations
            popSize = len(self.populations[i].individuals)
            probWeights = self.dispersalMatrix[i]
            migrationMap = random.choices(self.popAlias, weights=probWeights, k=popSize)
            #i.e. migrationMap => [popA,popB,popA,popA,popC,popA,popA,popC,popB,popA]
            migrants = []
            for indIndex, destPop in enumerate(migrationMap):
                if destPop != self.populations[i].id:
                    
                    migrant = self.populations[i].individuals[indIndex]
                    migrants.append(migrant)

                    destPopIndex = self.popAlias.index(destPop)
                    self.populations[destPopIndex].addIndividual(migrant)
            
            for migrant in migrants:
                self.populations[i].removeIndividual(migrant)

                    
    def startSimulation(self):
        for i in range(self.simulationSteps):
            print('Simulating step %d' % (i+1))
            self.migratePop()
    
    def plotPhenoFreq(self):
        pass

    def plotPopSizes(self):
        pass
        

In [74]:
phenotypes = ['green', 'blue', 'purple']
dispersalMatrix = [
    [0.6, 0.1, 0.3],
    [0.2, 0.7, 0.1],
    [0.1, 0.1, 0.8]]
steps = 3
testLandscape = Landscape(4, phenotypes, dispersalMatrix, steps)
testLandscape.startSimulation()
print('\nFinal landscape status:')
for pop in testLandscape.populations:
    print('Population: %s' % pop.id)
    for ind in pop.individuals:
        print('%s => %s' % (ind.id, ind.phenotype))

Simulation step 1
Simulation step 2
Simulation step 3

Final landscape status:
Population: popA
popA-2 => green
popA-3 => green
popB-2 => blue
popC-3 => purple
Population: popB
popB-3 => blue
popC-2 => purple
popC-1 => purple
Population: popC
popC-4 => purple
popA-1 => green
popA-4 => green
popB-1 => blue
popB-4 => blue
